In [0]:
/*
There are 4 types of ranking functions:-
1. row_number
     - deduplicate the data
     - unique row id
     - get record by specific category
2. rank
3. dense_rank
4. ntile

Syntax:
 <rank_function> over ([partition by <column-name>] order by <column-name>)

partition by is optional


*/
select custid,fname, prof, row_number() over(order by custid) as row_id from inceptez_catalog.inputdb.tblcusts;

select custid,fname, prof, row_number() over(partition by prof order by custid) as row_id from inceptez_catalog.inputdb.tblcusts;

--get without duplicates
with dedup as
(SELECT
  custid,
  fname,
  lname,
  age,
  prof,
  ROW_NUMBER() OVER (PARTITION BY custid order by custid) AS rn
FROM inceptez_catalog.inputdb.tblcusts)
select * from dedup where rn = 1;

select * from
(SELECT
  custid,
  fname,
  lname,
  age,
  prof,
  ROW_NUMBER() OVER (PARTITION BY custid order by custid) AS rn
FROM inceptez_catalog.inputdb.tblcusts) dedup
where rn = 1;


--get the max amount product sales at each state
select * from (
select *,row_number() over(partition by city order by amount desc) as rn from inceptez_catalog.inputdb.tbltxns where state='California')
where rn = 1;

-- short form by using the keyword qualify
select *,row_number() over(partition by state order by amount desc) as rn from inceptez_catalog.inputdb.tbltxns qualify rn = 1;

-- rank
select * from (
select *,rank() over(partition by city order by amount desc) as rn from inceptez_catalog.inputdb.tbltxns where state='California')
where rn = 1;

-- create sales data table

CREATE OR REPLACE TABLE salesdata (
  txnid STRING,
  custid STRING,
  state STRING,
  product STRING,
  amount DOUBLE
);

INSERT INTO salesdata VALUES
('T001', 'C001', 'CA', 'Laptop', 1000),
('T002', 'C002', 'CA', 'Phone', 900),
('T003', 'C002', 'CA', 'Phone', 1100),
('T004', 'C003', 'CA', 'Tablet', 900),
('T005', 'C004', 'NY', 'Laptop', 1200),
('T006', 'C005', 'NY', 'TV', 1200),
('T007', 'C006', 'NY', 'Phone', 900),
('T008', 'C007', 'NY', 'Tablet', 800);

select * from salesdata order by amount desc,custid desc;

WITH ranked AS (
  SELECT
    state,
    custid,
    SUM(amount) AS total_spend,
    DENSE_RANK() OVER (PARTITION BY state ORDER BY SUM(amount) DESC) AS rnk
  FROM salesdata
  GROUP BY state, custid
)
SELECT * FROM ranked;


-- row_number, rank and dense_rank difference

SELECT
  state,
  custid,
  SUM(amount) AS total_spend,
  ROW_NUMBER() OVER (
    PARTITION BY state
    ORDER BY SUM(amount) DESC
  ) AS rownum,
  RANK() OVER (
    PARTITION BY state
    ORDER BY SUM(amount) DESC
  ) AS rnk,
  DENSE_RANK() OVER (
    PARTITION BY state
    ORDER BY SUM(amount) DESC
  ) AS dense_rnk
FROM salesdata
GROUP BY state, custid
ORDER BY state, total_spend DESC;

/*
How to Interpret the Results

ROW_NUMBER() → Always unique sequence 1, 2, 3...

RANK() → Tied customers share same rank, gaps appear after ties
(Notice jump from 1 → 3 in NY)

DENSE_RANK() → Tied customers share same rank, no gaps
(Ranks go 1 → 2 → 3 continuously)
*/


WITH ranked AS (
  SELECT
    state,
    custid,
    SUM(amount) AS total_spend,
    ROW_NUMBER() OVER (PARTITION BY state,custid ORDER BY SUM(amount) DESC) AS rnk
  FROM salesdata
  GROUP BY state, custid
)
SELECT * FROM ranked; 

/*
NTILE() divides the result set into a specified number of groups (buckets) and assigns each row a bucket number starting from 1. 

*/

SELECT
  txnid,
  state,
  amount,
  NTILE(10) OVER (ORDER BY amount DESC) AS quartile
FROM 
(select * from inceptez_catalog.inputdb.tbltxns limit 100) order by quartile;

-- LAG() and LEAD() – Compare previous and next transactions

SELECT
  state,
  txndate,
  amount,
  LAG(amount, 2) OVER (PARTITION BY state ORDER BY txndate) AS prev_amount,
  LEAD(amount, 2) OVER (PARTITION BY state ORDER BY txndate) AS next_amount
FROM inceptez_catalog.inputdb.tbltxns

Databricks visualization. Run in Databricks to view.